In [39]:
"""
This first module will focus on loading the dataset from the generating step.
"""

from google.colab import files
uploaded = files.upload()

import pandas as pd
import copy

# Load the data
modified_data = pd.read_csv('modified_sysmon_data.csv')
generated_data = pd.read_csv('generated_sysmon_data111111.csv')

Saving generated_sysmon_data111111.csv to generated_sysmon_data111111.csv


In [40]:
"""
This module will begin postprocessing of data. It requires data structures from the preprocessing steps,
so it is important to run the same preprocessing code to obtain correct encodings. The code for the preprocessing
of the data is in the last module of this notebook for reference. This module will specifically build the
dictionaries that will be used to re-build the file paths.
"""

import ast
discrepancies = 0

# Function to convert a string representation of a list into an actual list
def string_to_list(string):
    try:
        return ast.literal_eval(string)
    except ValueError:
        # Return an empty list in case of an error (e.g., empty string)
        return []

tree_data = copy.deepcopy(modified_data)
# Initialize the dictionary
file_paths = {}
reverse_folderDict = {v: k for k, v in folderDict.items()}
reverse_directoryDict = {1: "Program Files", 2: "Program Files (x86)", 3: "Windows", 4: "Users"}
reverse_imageDict = {v: k for k, v in imageDict.items()}
# Drop the unneeded columns
tree_data = tree_data[["Encoded_CurrentDirectory", "Encoded_ImageDirectory"]]

# Apply this conversion to the relevant columns
tree_data['Encoded_CurrentDirectory'] = tree_data['Encoded_CurrentDirectory'].apply(string_to_list)
tree_data['Encoded_ImageDirectory'] = tree_data['Encoded_ImageDirectory'].apply(string_to_list)

for index, row in tree_data.iterrows():
    # Check for Encoded_ImageDirectory
    if row['Encoded_ImageDirectory']:
        last_token = row['Encoded_ImageDirectory'][-1]
        if last_token not in file_paths:
            file_paths[last_token] = row['Encoded_ImageDirectory']

    # Check for Encoded_CurrentDirectory
    if row['Encoded_CurrentDirectory']:
        last_token = row['Encoded_CurrentDirectory'][-1]
        if last_token not in file_paths:
            file_paths[last_token] = row['Encoded_CurrentDirectory']

# Pop Day and EventID from encoding_dicts
encoding_dicts.pop('Day', None)
encoding_dicts.pop('Event_ID', None)

print(file_paths)
print(reverse_folderDict)
print(reverse_imageDict)
print(encoding_dicts.keys())

{5: [1, 2, 3, 4, 5], 8: [6, 7, 8], 9: [6, 7, 8, 9], 33: [33], 10: [10], 108: [33, 108], 12: [11, 12], 14: [13, 14], 16: [1, 2, 15, 16], 110: [109, 110], 17: [11, 17], 18: [13, 18], 41: [3, 41], 112: [111, 112], 113: [113], 23: [33, 44, 45, 46, 47, 114, 22, 23], 115: [33, 44, 45, 46, 47, 114, 115], 118: [33, 116, 117, 47, 118], 24: [11, 24], 25: [6, 25], 26: [6, 25, 26], 121: [119, 120, 121], 27: [11, 27], 28: [13, 28], 36: [34, 122, 36], 29: [11, 29], 30: [13, 30], 31: [11, 31], 123: [34, 122, 123], 94: [124, 71, 90, 91, 92, 93, 94], 127: [119, 120, 125, 126, 35, 127], 131: [1, 2, 128, 129, 130, 131], 132: [33, 132], 134: [1, 2, 128, 129, 130, 131, 133, 134], 88: [88], 32: [32], 47: [33, 44, 45, 135, 47], 40: [37, 38, 39, 40], 80: [80], 42: [3, 41, 42], 43: [13, 43], 136: [33, 44, 45, 46, 47, 114, 136], 137: [33, 44, 45, 46, 47, 114, 137], 48: [48], 49: [11, 49], 50: [11, 50], 51: [11, 51], 52: [11, 52], 54: [3, 53, 8, 54], 138: [1, 2, 3, 138], 141: [3, 139, 140, 141], 55: [11, 55], 56

In [41]:
"""
This module will build the hardest variables to form from the data. These include the file
path strings, some of the randomized variables, and the time and date
"""

import random
import datetime

generated_data2 = copy.deepcopy(generated_data)

# This variable will keep track of major problems found in generated data
inconsistencies = 0

# Initialize lists to store the paths
current_pathList = []
image_pathList = []
imageList = []

# Function to generate a string of random hexadecimal characters of a given length
def random_hex(length):
    return ''.join(random.choice('0123456789abcdef') for _ in range(length))

for _, row in generated_data2.iterrows():
    current_path = []
    image_path = []
    image_folder = file_paths.get(row["Last_ImageDirectory"], [])
    current_folder = file_paths.get(row["Last_CurrentDirectory"], [])
    if not image_folder:
        print("Something has gone wrong")
    else:
        image_path.append("C:")
        if row["ImageDirectory_Type"] == 0:
            random_key = random.randint(1, 4)
            image_path.append(reverse_directoryDict[random_key])
            inconsistencies += 1
        else:
            image_path.append(reverse_directoryDict[row["ImageDirectory_Type"]])
        for folder in image_folder:
            image_path.append(reverse_folderDict[folder])
        image = reverse_imageDict[row["Encoded_Image"]] + '.exe'
        imageList.append(image)
        image_path.append(image)
    if current_folder and row["CurrentDirectory_Type"] != 0:
        current_path.append("C:")
        current_path.append(reverse_directoryDict[row["CurrentDirectory_Type"]])
        for folder in current_folder:
            current_path.append(reverse_folderDict[folder])
    current_pathList.append(current_path)
    image_pathList.append(image_path)

# Convert the path lists to strings and attach to dataframe
generated_data2['CurrentDirectory'] = ['\\'.join(path) for path in current_pathList]
generated_data2['Image'] = ['\\'.join(path) for path in image_pathList]
generated_data2['CurrentDirectory'] = generated_data2['CurrentDirectory'].apply(lambda x: x + '\\' if x else x)
generated_data2['OriginalFileName'] = imageList

# Add additional columns from already present data
generated_data2['ParentImage'] = generated_data2['Image']
generated_data2['CommandLine'] = '"' + generated_data2['Image'].astype(str) + '"'
generated_data2['ParentCommandLine'] = '"' + generated_data2['Image'].astype(str) + '"'

# See preprocessing script for floating numbers description
Guid_Start = "{8f201f84"
Guid_End = "070000004800}"

shaList = []

for _, row in generated_data2.iterrows():
    if not row['CurrentDirectory']:
        shaList.append("")
    else:
        random_sha = "SHA256=" + random_hex(64)
        shaList.append(random_sha)

generated_data2['Hashes'] = shaList


guidList = []
parentGuidList = []

for _, row in generated_data2.iterrows():
    # Constructing the process guid string
    random_guid = Guid_Start
    parent_random_guid = Guid_Start
    for _ in range(3):
        random_guid += '-' + random_hex(4)
        parent_random_guid += '-' + random_hex(4)
    random_guid += '-' + Guid_End
    parent_random_guid += '-' + Guid_End
    guidList.append(random_guid)
    parentGuidList.append(parent_random_guid)

# Generate random process Id's in the range observed from the dataset
generated_data2['ProcessId'] = [random.randint(1, 40000) for _ in range(len(generated_data2))]
generated_data2['ParentProcessId'] = [random.randint(1, 40000) for _ in range(len(generated_data2))]

# Add the guidList as a new column 'ProcessGuid'
generated_data2['ProcessGuid'] = guidList
generated_data2['ParentProcessGuid'] = parentGuidList

# Create a new list to store the combined date and time
date_and_time_list = []
utc_time_list = []


# Iterate over the rows of generated_data2
for _, row in generated_data2.iterrows():
    day = row['Day']
    time_str = row['Time']

    # Check for negative or invalid time and correct it if necessary
    try:
        # Try to parse the time, this will raise ValueError if time_str is invalid
        temp_time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S')
        if temp_time_obj.hour < 0:
            raise ValueError("Negative time value")
    except ValueError:
        # Handle the invalid time case
        inconsistencies += 1
        # Correcting the time to a default valid time
        time_str = '00:00:00'

    # Construct the datetime object with the corrected time
    date_time_obj = datetime.datetime.strptime(f'2023-11-{day} {time_str}', '%Y-%m-%d %H:%M:%S')

    # Add random seconds
    random_seconds = random.randint(0, 59)
    date_time_obj = date_time_obj.replace(second=random_seconds)

    # Format the datetime object for Date_and_Time column
    formatted_date_time = date_time_obj.strftime('%m/%d/%Y %I:%M:%S %p')
    date_and_time_list.append(formatted_date_time)

    # Create and format UtcTime
    utc_time_obj = date_time_obj + datetime.timedelta(hours=8)
    # Randomize seconds and milliseconds
    utc_seconds = random.randint(0, 59)
    utc_milliseconds = random.randint(0, 999)
    utc_time_obj = utc_time_obj.replace(second=utc_seconds, microsecond=utc_milliseconds * 1000)
    formatted_utc_time = utc_time_obj.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    utc_time_list.append(formatted_utc_time)

# Assign the lists to the DataFrame
generated_data2['Date_and_Time'] = date_and_time_list
generated_data2['UtcTime'] = utc_time_list

# Assign the lists to the DataFrame
generated_data2['Date_and_Time'] = date_and_time_list
generated_data2['UtcTime'] = utc_time_list

# Update variables to be None when CurrentDirectory is None
for index, row in generated_data2.iterrows():
    if not row['CurrentDirectory']:
        generated_data2.at[index, 'ParentProcessGuid'] = None
        generated_data2.at[index, 'Hashes'] = None
        generated_data2.at[index, 'CurrentDirectory'] = None
        generated_data2.at[index, 'OriginalFileName'] = None
        generated_data2.at[index, 'CommandLine'] = None
        generated_data2.at[index, 'ParentCommandLine'] = None
        generated_data2.at[index, 'ParentProcessId'] = None
        generated_data2.at[index, 'ParentImage'] = None

columns_to_drop = ['Time', 'Day', 'Last_CurrentDirectory', 'Last_ImageDirectory']
generated_data2.drop(columns=columns_to_drop, axis=1, inplace=True)

# Verify the DataFrame with new columns
print(generated_data2.head())


   Event_ID  TerminalSessionId  FileVersion  LogonId  LogonGuid  Company  \
0         1                  0            2        0          0        0   
1         1                  0            1        0          0        0   
2         1                  0            1        0          0        0   
3         1                  0            1        0          0        0   
4         1                  0            1        0          0        0   

   Product  Description  IntegrityLevel  ImageDirectory_Type  ...  \
0        0            2               0                    2  ...   
1        0            1               0                    1  ...   
2        0            2               0                    2  ...   
3        0            2               0                    1  ...   
4        0            1               0                    2  ...   

   ParentImage  CommandLine  ParentCommandLine  Hashes  ProcessId  \
0         None         None               None    None     

In [42]:
"""
This module will reconstruct all of the easier variables. These are the variables encoded by encoding_dicts.
Then, it will rectify the primary inconsistency of the data - that the data should exist with a subset of
variables that all will either have a value or all not have a value. The governing factor for this will be
CurrentDirectory. If CurrentDirectory does not exist, many of the variables will be set to None. If it does
exist but a variable is equal to None, a value must be found. An inconsistency count will be kept that will
be used later to evaluate one GAN's performance against another.
"""

generated_data3 = copy.deepcopy(generated_data2)

for column, encoding_dict in encoding_dicts.items():
    # Invert the encoding dictionary to map from numbers back to strings
    inverted_dict = {v: k for k, v in encoding_dict.items()}

    # Apply the conversion to the DataFrame column
    if column in generated_data3.columns:
        generated_data3[column] = generated_data3[column].map(inverted_dict)

columns_to_drop = ['Encoded_Image', 'ParentImage_Encoded', 'CurrentDirectory_Type', 'ImageDirectory_Type']
generated_data3.drop(columns=columns_to_drop, axis=1, inplace=True)

variables_to_check = encoding_dicts.keys()

# This loop is where inconsistencies are check for in the data
for index, row in generated_data3.iterrows():
    for variable in variables_to_check:
        if variable == 'User':
            continue
        if pd.isna(row[variable]) and row['CurrentDirectory'] is not None:
            inconsistencies += 1
            # Select a random key from encoding_dicts[variable]
            keys = list(encoding_dicts[variable].keys())
            new_value = encoding_dicts[variable][keys[random.randint(0, len(keys) - 1)]]
            # Ensure the new value is not NaN
            while pd.isna(new_value):
                new_value = encoding_dicts[variable][keys[random.randint(0, len(keys) - 1)]]
            # Assign the new value to the DataFrame
            generated_data3.at[index, variable] = new_value
        elif not pd.isna(row[variable]) and row['CurrentDirectory'] is None:
            inconsistencies += 1
            print("%s , %s\n" %(index, variable))
            generated_data3.at[index, variable] = None

for index, row in generated_data3.iterrows():
    if row['Event_ID'] == 1 and row['CurrentDirectory'] is None:
        inconsistencies += 1
        generated_data3.at[index, 'Event_ID'] = 5
    elif row['Event_ID'] == 5 and row['CurrentDirectory'] is not None:
        inconsistencies += 1
        generated_data3.at[index, 'Event_ID'] = 1

print(generated_data3.head())
print(inconsistencies)


0 , TerminalSessionId

0 , FileVersion

0 , LogonId

0 , LogonGuid

0 , Company

0 , Product

0 , Description

0 , IntegrityLevel

0 , ParentUser

1 , TerminalSessionId

1 , FileVersion

1 , LogonId

1 , LogonGuid

1 , Company

1 , Product

1 , Description

1 , IntegrityLevel

1 , ParentUser

2 , TerminalSessionId

2 , FileVersion

2 , LogonId

2 , LogonGuid

2 , Company

2 , Product

2 , Description

2 , IntegrityLevel

2 , ParentUser

3 , TerminalSessionId

3 , FileVersion

3 , LogonId

3 , LogonGuid

3 , Company

3 , Product

3 , Description

3 , IntegrityLevel

3 , ParentUser

4 , TerminalSessionId

4 , FileVersion

4 , LogonId

4 , LogonGuid

4 , Company

4 , Product

4 , Description

4 , IntegrityLevel

4 , ParentUser

5 , TerminalSessionId

5 , FileVersion

5 , LogonId

5 , LogonGuid

5 , Company

5 , Product

5 , Description

5 , IntegrityLevel

5 , ParentUser

6 , TerminalSessionId

6 , FileVersion

6 , LogonId

6 , LogonGuid

6 , Company

6 , Product

6 , Description

6 , Int

In [9]:
"""
Final Postprocessing! In this final module, we take the data and convert it directly back to the form
from which we got it. This will involve creating a new dataframe and placing the data into the dataframe
in a very untidy manner to replicate its original form
"""

generated_data4 = copy.deepcopy(generated_data3)

def terminate_build(index):
    row = generated_data4.loc[index]

    # Build the string in the same format as Sysmon data
    process_string = f'Process terminated:\n'
    process_string += f"RuleName: -\n"
    process_string += f"UtcTime: {row['UtcTime']}\n"
    process_string += f"ProcessGuid: {row['ProcessGuid']}\n"
    process_string += f"ProcessId: {row['ProcessId']}\n"
    process_string += f"Image: {row['Image']}\n"
    process_string += f"User: {row['User']}"

    return process_string

def create_build(index):
    row = generated_data4.loc[index]

    # Build the initial part of the string in the same format as small_build
    process_string = f'Process Create:\n'
    process_string += f"RuleName: -\n"
    process_string += f"UtcTime: {row['UtcTime']}\n"
    process_string += f"ProcessGuid: {row['ProcessGuid']}\n"
    process_string += f"ProcessId: {row['ProcessId']}\n"
    process_string += f"Image: {row['Image']}\n"

    # Append additional information
    process_string += f"FileVersion: {row['FileVersion']}\n"
    process_string += f"Description: {row['Description']}\n"
    process_string += f"Product: {row['Product']}\n"
    process_string += f"Company: {row['Company']}\n"
    process_string += f"OriginalFileName: {row['OriginalFileName']}\n"
    process_string += f"CommandLine: {row['CommandLine']}\n"
    process_string += f"CurrentDirectory: {row['CurrentDirectory']}\n"
    process_string += f"User: {row['User']}\n"
    process_string += f"LogonGuid: {row['LogonGuid']}\n"
    process_string += f"LogonId: {row['LogonId']}\n"

    # TerminalSessionId formatted as an integer
    terminal_session_id = int(row['TerminalSessionId'])
    process_string += f"TerminalSessionId: {terminal_session_id}\n"

    process_string += f"IntegrityLevel: {row['IntegrityLevel']}\n"
    process_string += f"Hashes: {row['Hashes']}\n"
    process_string += f"ParentProcessGuid: {row['ParentProcessGuid']}\n"
    parent_process_id = int(row['ParentProcessId'])
    process_string += f"ParentProcessId: {parent_process_id}\n"
    process_string += f"ParentImage: {row['ParentImage']}\n"
    process_string += f"ParentCommandLine: {row['ParentCommandLine']}\n"
    process_string += f"ParentUser: {row['ParentUser']}"

    return process_string


# Create an empty Dataframe with the same number of rows as our generated data
final_data = pd.DataFrame(index=generated_data4.index)

# Populate the DataFrame
final_data['Level'] = 'Information'
final_data['Date and Time'] = generated_data4['Date_and_Time']
final_data['Source'] = 'Microsoft-Windows-Sysmon'
final_data['Event ID'] = generated_data4['Event_ID']

for index, row in generated_data4.iterrows():
    if row['Event_ID'] == 5:
        final_data.at[index, "Task Category"] = 'Process terminated (rule: ProcessTerminate)'
    elif row['Event_ID'] == 1:
        final_data.at[index, "Task Category"] = 'Process Create (rule: ProcessCreate)'

final_data['deleteName'] = ''

for index, row in generated_data4.iterrows():
    if row['CurrentDirectory'] is None:
        final_data.at[index, "deleteName"] = terminate_build(index)
    else:
        final_data.at[index, "deleteName"] = create_build(index)

# Display the information one last time
print(final_data)


          Level           Date and Time                    Source  Event ID  \
0   Information  11/11/2023 08:34:35 PM  Microsoft-Windows-Sysmon         1   
1   Information  11/12/2023 04:48:01 PM  Microsoft-Windows-Sysmon         1   
2   Information  11/13/2023 08:49:20 AM  Microsoft-Windows-Sysmon         1   
3   Information  11/13/2023 09:51:23 AM  Microsoft-Windows-Sysmon         5   
4   Information  11/12/2023 01:14:11 PM  Microsoft-Windows-Sysmon         5   
..          ...                     ...                       ...       ...   
95  Information  11/11/2023 04:12:41 PM  Microsoft-Windows-Sysmon         5   
96  Information  11/12/2023 12:54:50 PM  Microsoft-Windows-Sysmon         5   
97  Information  11/13/2023 08:16:06 AM  Microsoft-Windows-Sysmon         1   
98  Information  11/11/2023 03:33:03 PM  Microsoft-Windows-Sysmon         5   
99  Information  11/12/2023 10:18:02 AM  Microsoft-Windows-Sysmon         1   

                                  Task Category  \


In [83]:
"""
And now, we can download our newly generated data
"""

final_data.to_csv('final_data.csv', index=False)
files.download('final_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
"""
This module contains all of the code needed for the preprocessing of the data. All of this code is contained in
the GeneratingData_PreProcessing notebook, but is repeated here for reference.
"""


"""
First, we must begin to prepare the data and simpligy it so that we may train a model on it.
The first step will be simplifying the date and time of the features
"""

from google.colab import files
uploaded = files.upload()

import pandas as pd
import copy

# Load the data
sysmon_data = pd.read_csv('tidy_sysmon_data.csv')

# Drop the 'UtcTime' feature
sysmon_data.drop('UtcTime', axis=1, inplace=True)

# Transform 'Date_and_Time' into 'Day' and 'Time'
sysmon_data['Day'] = pd.to_datetime(sysmon_data['Date_and_Time']).dt.day
sysmon_data['Time'] = pd.to_datetime(sysmon_data['Date_and_Time']).dt.time

# Drop the original 'Date_and_Time' feature
sysmon_data.drop('Date_and_Time', axis=1, inplace=True)



# Display the first few rows of the DataFrame to verify changes
# print(sysmon_data.head())
"""
Next, we will continue to prune the features. From the dataset, some of the features may be
dropped due to uniformality, redundancy, or randomization. Following that, we will convert
some of the features into a categorical nature to make training easier. Note, this can only
be done on features where there are a small set of possibilities, or else we run the risk
of vastly increasing the dimensionality of our data.

The reasons for the drop variables are as follows:
Level - This is the same for all observations and thus is not needed.
Source - This is the same for all observations and thus is not needed.
RuleName - This is the same for all observations and thus is not needed.
Task_Category - This information corresponds exactly to Event_Id and is therefore redundant.
Hashes - These are assumed to be random and thus nothing will be learned from this information.
OriginalFileName - This corresponds exactly to the basename of Image and is thus rendendant.
ProcessGuid and ParentProcessGuid - Both of these consist of five token hexadecimal strings \
separated by dashes. The first and last token are consisten throughout the dataset, and the \
middle three tokens consist of randomized 4-character strings. Thus, the first and last tokens \
will be saved for remaking logs later and the two variables will be dropped from training.
"""

# Saving this to a new variable makes it easer to re-run individual modules
sysmon_data1 = copy.deepcopy(sysmon_data)

# Drop the unneeded columns
columns_to_drop = ['Level', 'Source', 'Task_Category', 'Hashes', 'OriginalFileName', \
                   'RuleName', 'ProcessGuid', 'ParentProcessGuid']
sysmon_data1.drop(columns_to_drop, axis=1, inplace=True)

# Categorical variables for encoding
categorical_columns = ['Event_ID', 'TerminalSessionId', 'FileVersion', 'LogonId', \
                       'LogonGuid', 'Day', 'Company', 'Product', 'Description', \
                       'IntegrityLevel', 'ParentUser', 'User']

# These were all found through visual observation of the data and are being labeled here
# for future reference
Guid_Start = "{8f201f84"
Guid_End = "070000004800}"


# Dictionary for storing mappings
encoding_dicts = {}

for column in categorical_columns:
    # Create a mapping for each unique value in the column
    unique_values = sysmon_data1[column].unique()
    encoding_dict = {value: idx for idx, value in enumerate(unique_values)}

    # Store the mapping dictionary
    encoding_dicts[column] = encoding_dict

    # Apply the mapping to the column
    sysmon_data1[column] = sysmon_data1[column].map(encoding_dict)

# Print the encoding dictionaries
for column, d in encoding_dicts.items():
    print(f"Encoding for {column}: {d}")


# Verify the changes
# print(sysmon_data1.head())
"""
This section will be tokenize the string variables which cannot be otherwise dropped or made categorical.
The token string elements will be the most difficult portion to generate data for, so it is important
that the data is well-understood
"""

sysmon_data2 = copy.deepcopy(sysmon_data1)

# Function to remove exectuable command from a given command line
def remove_command_token(string):
    if not isinstance(string,str):
        return None

    # Find the indices of the first two double quotes
    first_quote_index = string.find('"')
    second_quote_index = string.find('"', first_quote_index + 1)

    # If both double quotes are found, slice the string from the character after the second quote
    if first_quote_index != -1 and second_quote_index != -1:
        return string[second_quote_index + 2:]

    # If double quotes not found, find the first space and slice the string from there
    first_space_index = string.find(' ')
    if first_space_index != -1:
        return string[first_space_index + 1:]

    return string

# Function to tokenize a string based on a given delimiter
def tokenize_string(string, delimiter):
    # Check if the input is a string
    if isinstance(string, str):
        return string.split(delimiter)
    # Return an empty list if not a string
    else:
        return []

# Function to get the directory type and update the token lists
def get_directory_branch(tokens, directory_dict):

    if tokens is None:
        return 0, None  # 0 for None values
    if not tokens:
        return 0, tokens  # Empty list remains unchanged

    # Get the directory type
    directory_type = directory_dict.get(tokens[0].lower(), 0)  # Default to 0 if not found

    # Remove the first token (and second if first is 'Users')
    if tokens[0] == 'Users' and len(tokens) > 1:
        updated_tokens = tokens[2:]  # Remove first two tokens
    else:
        updated_tokens = tokens[1:]  # Remove first token

    return directory_type, updated_tokens

# Function to remove '.exe' from the end of a string if it exists
def remove_exe_extension(string):
    if string is None:
        return None
    if string.lower().endswith('.exe'):
        return string[:-4]  # Remove the last 4 characters '.exe'
    return string


# Removing the command from a command line prevents tokenization problems using ' ' char
sysmon_data2['CommandLine_Tokens'] = sysmon_data2['CommandLine'].apply\
            (lambda x: tokenize_string(remove_command_token(x), ' '))
sysmon_data2['ParentCommandLine_Tokens'] = sysmon_data2['ParentCommandLine'].apply\
            (lambda x: tokenize_string(remove_command_token(x), ' '))

# # Applying tokenization to the specified columns
sysmon_data2['Image_Tokens'] = \
sysmon_data2['Image'].apply(lambda x: tokenize_string(x, '\\'))
sysmon_data2['CommandLine_Args'] = \
sysmon_data2['CommandLine'].apply(lambda x: tokenize_string(x, ' '))
sysmon_data2['CurrentDirectory_Tokens'] = \
sysmon_data2['CurrentDirectory'].apply(lambda x: tokenize_string(x, '\\'))
sysmon_data2['ParentImage_Tokens'] = \
sysmon_data2['ParentImage'].apply(lambda x: tokenize_string(x, '\\'))
sysmon_data2['ParentCommandLine_Args'] = \
sysmon_data2['ParentCommandLine'].apply(lambda x: tokenize_string(x, ' '))

print(sysmon_data2.head())

# Update Image to only contain the last token of Image_Tokens
sysmon_data2['Image'] = sysmon_data2['Image_Tokens'].apply(lambda tokens: tokens[-1] if tokens else None)

# Update Image_Directory to exclude the first and last token
# The first token will always be 'C:', and the last token is saved to Image
sysmon_data2['Image_Directory'] = sysmon_data2['Image_Tokens'].apply(lambda tokens: tokens[1:-1] if tokens else None)

# Update CurrentDirectory_Tokens to exclude the first token
# The first token will always be 'C:'
sysmon_data2['CurrentDirectory_Tokens'] = sysmon_data2['CurrentDirectory_Tokens'].apply(lambda tokens: tokens[1:] if tokens else None)

# Remove the first token from CommandLine_Tokens
# This token can be gathered from Image_Directory and Image
sysmon_data2['CommandLine_Args'] = sysmon_data2['CommandLine_Args'].apply(lambda tokens: tokens[1:] if tokens else None)

# Update ParentImage to only contain the last token of ParentImage_Tokens
# This token will contain just the parent executable
sysmon_data2['ParentImage'] = sysmon_data2['ParentImage_Tokens'].apply(lambda tokens: tokens[-1] if tokens else None)

# Remove the first token from ParentCommandLine_Tokens
# This token can be gathered from Image_Directory and Image
sysmon_data2['ParentCommandLine_Args'] = sysmon_data2['ParentCommandLine_Args'].apply(lambda tokens: tokens[1:] if tokens else None)


# Creating the dictionary for directory mappings
directoryDict = {"program files": 1, "program files (x86)": 2, "windows": 3, "users": 4}

# Apply the function to Image_Directory and CurrentDirectory_Tokens
sysmon_data2['ImageDirectory_Type'], sysmon_data2['Image_Directory'] = zip(*sysmon_data2['Image_Directory'].apply(lambda x: get_directory_branch(x, directoryDict)))
sysmon_data2['CurrentDirectory_Type'], sysmon_data2['CurrentDirectory_Tokens'] = zip(*sysmon_data2['CurrentDirectory_Tokens'].apply(lambda x: get_directory_branch(x, directoryDict)))
# Final Preprocessing Cleanup Steps
# Removes the empty string present as the final token of CurrentDirectory_Tokens
sysmon_data2['CurrentDirectory_Tokens'] = sysmon_data2['CurrentDirectory_Tokens'].apply(lambda tokens: tokens[0:-1] if tokens else None)
# Removes the .exe extension from Image
sysmon_data2['Image'] = sysmon_data2['Image'].apply(remove_exe_extension)
# Removes the .exe extension from ParentImage
sysmon_data2['ParentImage'] = sysmon_data2['ParentImage'].apply(remove_exe_extension)
# Defines and drops the columns that are no longer needed
columns_to_drop = ['Image_Tokens', 'ParentImage_Tokens', 'CurrentDirectory', 'CommandLine', 'ParentCommandLine', 'CommandLine_Args', 'ParentCommandLine_Args']
sysmon_data2.drop(columns=columns_to_drop, axis=1, inplace=True)

"""
After completing the previous module, Image and ParentImage may also be encoded as categorical
values. After this, we begin the final stage of encoding the tokens of CurrentDirectory_Tokens
and Image_Directory as lists of numbers. This may be the final encoding before beginning to build
a model, as CommandLine_Tokens and ParentCommandLine_Tokens are too diverse to build a model on
within the scope of this project.

Note: I am leaving some of the variables that will not be used in training intact. While this
module condes the folders, it does not accurately capture the entire structure of the file
system, and so I am leaving the Image_Directory and CurrentDirectory_Tokens intact at this
stage. These will be removed before training or if a better encoding method is found. This will
represent a point in processing where nearly all original information about the dataset remains
intact.
"""

sysmon_data3 = copy.deepcopy(sysmon_data2)
imageDict = {} # Initialize an empty dictionary for executable encodings
folderDict = {}  # Initialize an empty dictionary for folder encodings

def encode_image(image, encoding_dict):
    if image not in encoding_dict:
        # Assign a new encoding: length of the dictionary + 1
        encoding_dict[image] = len(encoding_dict) + 1
    return encoding_dict[image]

def encode_folders(folder_list, encoding_dict):
    encoded_list = []
    for folder in folder_list:
        if folder:  # Check if folder is not None or empty
            if folder not in encoding_dict:
                encoding_dict[folder] = len(encoding_dict) + 1
            encoded_list.append(encoding_dict[folder])
    return encoded_list

# Encode the executables from the dataset
sysmon_data3['Image_Encoded'] = sysmon_data3['Image'].apply(lambda x: encode_image(x, imageDict))
sysmon_data3['ParentImage_Encoded'] = sysmon_data3['ParentImage'].apply(lambda x: encode_image(x, imageDict))

# Eliminate None values from the dataset
sysmon_data3['CurrentDirectory_Tokens'] = sysmon_data3['CurrentDirectory_Tokens'].apply(lambda x: x if x is not None else [])
sysmon_data3['ParentProcessId'] = sysmon_data3['ParentProcessId'].fillna(0)

# Encoding the folder structures
sysmon_data3['Encoded_CurrentDirectory'] = sysmon_data3['CurrentDirectory_Tokens'].apply(lambda x: encode_folders(x, folderDict))
sysmon_data3['Encoded_ImageDirectory'] = sysmon_data3['Image_Directory'].apply(lambda x: encode_folders(x, folderDict))

# Defines and drops the columns that are no longer needed
columns_to_drop = ['Image', 'ParentImage']
sysmon_data3.drop(columns=columns_to_drop, axis=1, inplace=True)

print(imageDict)
print(folderDict)


Saving tidy_sysmon_data.csv to tidy_sysmon_data.csv
Encoding for Event_ID: {5: 0, 1: 1}
Encoding for TerminalSessionId: {2.0: 0, 0.0: 1, nan: 2, 3.0: 3}
Encoding for FileVersion: {'1.6.00.29964': 0, '119.0.6045.106': 1, '10.0.19041.3570 (WinBuild.160101.0800)': 2, nan: 3, '10.19.0 Build 19950': 4, '23.214.1015.0001': 5, '1.3.127.15': 6, '10.0.19041.3562 (WinBuild.160101.0800)': 7, '1, 0, 0, 2': 8, '10.0.19041.1 (WinBuild.160101.0800)': 9, '1.0.4.2': 10, '7.0.19041.3570 (WinBuild.160101.0800)': 11, '1.3.4.0': 12, '1.3.1.0': 13, '1.3.35.451': 14, '1.3.36.311': 15, '16.0.16924.20124': 16, '16.0.14326.21738': 17, '3.7.2181.36443': 18, '5.2.35.0': 19, '1.60.3467.0': 20, '1.3.2.1': 21, '16.0.16924.20054': 22, '0.14.9.0': 23, '4.2.1608.0': 24, '2.90': 25, '1.17.9.0': 26, '10.0.19041.3623 (WinBuild.160101.0800)': 27, '121.9202.5630.0': 28, '119.0.2151.44': 29, '2.0.0.5': 30, '4.18.23100.2009 (8fcbd1c22d82af16ba34560e1a70591413e88d17)': 31, '1.0.2.0': 32, '1.0.0.4': 33, '1.0.0.8': 34, '2.0.0.7'